<a href="https://colab.research.google.com/github/ANAVDUTT/Analyzing-Pump-and-Dump-Schemes/blob/main/Copy_of_ts_pump_dump_explore_20220426.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
# [INPUT REQUIRED]: Authorize access to Google gDrive

# Connect this Notebook to your permanent Google Drive
#   so all generated output is saved to permanent storage there

try:
  from google.colab import drive
  IN_COLAB=True
except:
  IN_COLAB=False

if IN_COLAB:
  print("Attempting to attach your Google gDrive to this Colab Jupyter Notebook")
  drive.mount('/gdrive', force_remount=True)
else:
  print("Your Google gDrive is attached to this Colab Jupyter Notebook")

Attempting to attach your Google gDrive to this Colab Jupyter Notebook
Mounted at /gdrive


In [ ]:
%cd /gdrive/MyDrive/Code/senior_seminar/
!ls -altr

/gdrive/MyDrive/Code/senior_seminar
total 313977
-rw------- 1 root root      5592 Oct  7  2020  catfish.csv
-rw------- 1 root root      9186 Mar  9  2021  TimeSeriesExpedia.csv
-rw------- 1 root root    936364 Mar 15  2021  Anomaly_Detection.ipynb
-rw------- 1 root root    208250 Sep 29  2021  BTC-USD.csv
-rw------- 1 root root       143 Oct  4  2021 'Project notes.gdoc'
-rw------- 1 root root     41774 Nov  5 17:36  10yrUSTreas.csv
-rw------- 1 root root     66808 Nov  5 17:38  SP_500.csv
-rw------- 1 root root    681662 Nov 15 17:54 'bitcoin-price-prediction-arima-xgboost-lstm-fbprop (1).ipynb'
-rw------- 1 root root 317526599 Nov 25 18:40  BTC.csv
-rw------- 1 root root    133058 Dec  6 17:06  TSLA.csv
-rw------- 1 root root    154711 Dec  6 17:21  NSDQ.csv
-rw------- 1 root root   1646919 Feb 28 02:24 'Copy of Bitcoin Forecasting'
-rw------- 1 root root     15427 Apr 27 16:01  SIMPLXPAP_BO.csv
-rw------- 1 root root     82891 May  2 23:14  BALFC_BO.csv


In [ ]:
# Core Python Libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime

%matplotlib inline
plt.rcParams["figure.figsize"] = (20,10) # plt.rcParamsDefault["figure.figsize"]

import plotly.express as px 
import plotly.graph_objs as go

from statsmodels.nonparametric.smoothers_lowess import lowess


/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [ ]:
!ls -1 *.csv

10yrUSTreas.csv
BALFC_BO.csv
BTC.csv
BTC-USD.csv
catfish.csv
NSDQ.csv
SIMPLXPAP_BO.csv
SP_500.csv
TimeSeriesExpedia.csv
TSLA.csv


In [ ]:
# Only run once

# !unzip features_15S.csv.gz

In [ ]:
!ls -altr data_samples

ls: cannot access 'data_samples': No such file or directory


# Read Datafile

## Read Raw Data

In [ ]:
!ls -altr

total 313977
-rw------- 1 root root      5592 Oct  7  2020  catfish.csv
-rw------- 1 root root      9186 Mar  9  2021  TimeSeriesExpedia.csv
-rw------- 1 root root    936364 Mar 15  2021  Anomaly_Detection.ipynb
-rw------- 1 root root    208250 Sep 29  2021  BTC-USD.csv
-rw------- 1 root root       143 Oct  4  2021 'Project notes.gdoc'
-rw------- 1 root root     41774 Nov  5 17:36  10yrUSTreas.csv
-rw------- 1 root root     66808 Nov  5 17:38  SP_500.csv
-rw------- 1 root root    681662 Nov 15 17:54 'bitcoin-price-prediction-arima-xgboost-lstm-fbprop (1).ipynb'
-rw------- 1 root root 317526599 Nov 25 18:40  BTC.csv
-rw------- 1 root root    133058 Dec  6 17:06  TSLA.csv
-rw------- 1 root root    154711 Dec  6 17:21  NSDQ.csv
-rw------- 1 root root   1646919 Feb 28 02:24 'Copy of Bitcoin Forecasting'
-rw------- 1 root root     15427 Apr 27 16:01  SIMPLXPAP_BO.csv
-rw------- 1 root root     82891 May  2 23:14  BALFC_BO.csv


In [ ]:
"""
!mv SIMPLXPAP.BO.csv SIMPLXPAP_BO.csv
!mv BALFC.BO.csv BALFC_BO.csv
!mv ADANIPOWER.NS.csv ADANIPOWER_NS.csv
"""

'\n!mv SIMPLXPAP.BO.csv SIMPLXPAP_BO.csv\n!mv BALFC.BO.csv BALFC_BO.csv\n!mv ADANIPOWER.NS.csv ADANIPOWER_NS.csv\n'

In [ ]:
# Stock Details

# stock_name = 'ADANIPOWER_NS'
stock_name = 'BALFC_BO'
# stock_name = 'SIMPLXPAP_BO'

#stock_ts_ls = ['7NR_BO','ADANIPOWER_NS','ADINATH_BO','AGROPHOS_NS','BALFC_BO','BANASFN_BO','SIMPLXPAP_BO','TTML_NS']
#stock_name = stock_ts_ls[5]

stock_datafile = f'{stock_name}.csv'
# stock_datapath = f'./data_samples/{stock_datafile}'
stock_datapath = f'./{stock_datafile}'

In [ ]:
df = pd.read_csv(stock_datapath , parse_dates=['Date'], infer_datetime_format=True, index_col=0, dayfirst=True)

# Remove Space in Col Name
df.rename(columns={'Adj Close':'Adjusted_Close'}, inplace=True)

# Index using Date
# df.index = df['Date']
# df.drop(columns=['Date'], inplace=True)
# df.index = pd.DatetimeIndex(df.index)

# df = df.reindex(df.index, fill_value=np.nan)
# df = df.reindex(df.index, fill_value=0)

df.head()
df.info()

,Open,High,Low,Close,Adjusted_Close,Volume
Date,,,,,,
2017-04-13,68.550003,68.599998,66.250000,67.000000,65.432434,6960
2017-04-17,69.000000,69.949997,67.650002,68.500000,66.897339,12000
2017-04-18,69.099998,70.000000,69.099998,69.099998,67.483299,9726
2017-04-19,69.000000,69.849998,69.000000,69.800003,68.166931,10100
2017-04-20,69.000000,70.000000,68.000000,69.599998,67.971603,7642


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1236 entries, 2017-04-13 to 2022-04-12
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Open            1236 non-null   float64
 1   High            1236 non-null   float64
 2   Low             1236 non-null   float64
 3   Close           1236 non-null   float64
 4   Adjusted_Close  1236 non-null   float64
 5   Volume          1236 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 67.6 KB


In [ ]:
"""
df = pd.read_csv(stock_datapath , parse_dates=['Date'], infer_datetime_format=True, index_col=0, dayfirst=True)

# Remove Space in Col Name
df.rename(columns={'Adj Close':'Adjusted_Close'}, inplace=True)

# Index using Date
df.index = df['Date']
df.drop(columns=['Date'], inplace=True)
# s.index = pd.DatetimeIndex(s.index)

df.head()
df.info()
"""

"\ndf = pd.read_csv(stock_datapath , parse_dates=['Date'], infer_datetime_format=True, index_col=0, dayfirst=True)\n\n# Remove Space in Col Name\ndf.rename(columns={'Adj Close':'Adjusted_Close'}, inplace=True)\n\n# Index using Date\ndf.index = df['Date']\ndf.drop(columns=['Date'], inplace=True)\n# s.index = pd.DatetimeIndex(s.index)\n\ndf.head()\ndf.info()\n"

## Drop Unnecessary Columns

In [ ]:
# cols_keep_ls = ['Adjusted_Close','High','Low']
cols_keep_ls = df.columns
cols_drop_ls = list(set(df.columns) - set(cols_keep_ls))
df.drop(columns=cols_drop_ls, inplace=True) 
df.head()

,Open,High,Low,Close,Adjusted_Close,Volume
Date,,,,,,
2017-04-13,68.550003,68.599998,66.250000,67.000000,65.432434,6960
2017-04-17,69.000000,69.949997,67.650002,68.500000,66.897339,12000
2017-04-18,69.099998,70.000000,69.099998,69.099998,67.483299,9726
2017-04-19,69.000000,69.849998,69.000000,69.800003,68.166931,10100
2017-04-20,69.000000,70.000000,68.000000,69.599998,67.971603,7642


## Drop Rows with Nulls

In [ ]:
df.head()
df.tail()
df.info()

,Open,High,Low,Close,Adjusted_Close,Volume
Date,,,,,,
2017-04-13,68.550003,68.599998,66.250000,67.000000,65.432434,6960
2017-04-17,69.000000,69.949997,67.650002,68.500000,66.897339,12000
2017-04-18,69.099998,70.000000,69.099998,69.099998,67.483299,9726
2017-04-19,69.000000,69.849998,69.000000,69.800003,68.166931,10100
2017-04-20,69.000000,70.000000,68.000000,69.599998,67.971603,7642


,Open,High,Low,Close,Adjusted_Close,Volume
Date,,,,,,
2022-04-06,41.599998,41.599998,41.020000,41.599998,41.599998,44120
2022-04-07,43.610001,43.680000,39.520000,43.400002,43.400002,126920
2022-04-08,44.799999,45.220001,41.259998,43.950001,43.950001,54465
2022-04-11,45.540001,45.540001,41.759998,44.369999,44.369999,444175
2022-04-12,46.400002,46.549999,45.900002,46.500000,46.500000,46349


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1236 entries, 2017-04-13 to 2022-04-12
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Open            1236 non-null   float64
 1   High            1236 non-null   float64
 2   Low             1236 non-null   float64
 3   Close           1236 non-null   float64
 4   Adjusted_Close  1236 non-null   float64
 5   Volume          1236 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 67.6 KB


In [ ]:
# How many Nulls per column

row_null_ct = df.isnull().sum().sum()
row_all_ct = df.shape[0]

row_per_null_ct = row_null_ct/row_all_ct

if row_null_ct > 0:

  test_str = str(df.isnull().sum())
  print(f'\nNulls by Column:\n----------------\n{test_str}\n')

  print(f'\nRow Count with any Null: [{row_null_ct}]')

  print(f'\n\nRow Count Total: [{row_all_ct}]')

  print(f'\n\nPercent of Rows with Null: [{100*row_per_null_ct:.3f}%]')

else:

  print('There are no Nulls in this dataset')

There are no Nulls in this dataset


In [ ]:
# Set %Rows with Nulls Threshold and
#   drop any row with Null if below threshold

DROP_NULL_ROW_PER = 5  # 0.485  # Percent of rows with any Nulls that can be safely dropped
 
if DROP_NULL_ROW_PER > 100*row_per_null_ct:
  print(f'Can safely drop all Rows with Nulls (Threshold={DROP_NULL_ROW_PER}% > {100*row_per_null_ct:.3f}%)')
  print(f'  BEFORE: DataFrame.shape before NULL Rows Dropped: {df.shape[0]}')
  df = df.dropna()
  print(f'  AFTER: DataFrame.shape after Null Rows Dropped: {df.shape[0]}')
  # row_nonull_ct = df.shape[0]
  # print(f'\n  Dropped {row_wnull_ct-row_all_ct}') 

else:
  print(f'WARNING: Rows with Null Count exceeds Threshold (Threshold={DROP_NULL_ROW_PER}% < {100*row_per_null_ct:.3f}%)')
  print('\nSUGGESTION:')
  print('  (a) Drop Columns that are not of interest and try again')
  print('  (b) If (1) does not work, use DataFrame imputation methods (linear, ff, fb) to fill in missing values')


Can safely drop all Rows with Nulls (Threshold=5% > 0.000%)
  BEFORE: DataFrame.shape before NULL Rows Dropped: 1236
  AFTER: DataFrame.shape after Null Rows Dropped: 1236


In [ ]:
df.shape

(1236, 6)

## Impute Missing Dates

In [ ]:
df.head()

,Open,High,Low,Close,Adjusted_Close,Volume
Date,,,,,,
2017-04-13,68.550003,68.599998,66.250000,67.000000,65.432434,6960
2017-04-17,69.000000,69.949997,67.650002,68.500000,66.897339,12000
2017-04-18,69.099998,70.000000,69.099998,69.099998,67.483299,9726
2017-04-19,69.000000,69.849998,69.000000,69.800003,68.166931,10100
2017-04-20,69.000000,70.000000,68.000000,69.599998,67.971603,7642


In [ ]:
# s = s.reindex(idx, fill_value=0)

In [ ]:
df.index.min()

Timestamp('2017-04-13 00:00:00')

In [ ]:
idx = pd.date_range(df.index.min(), df.index.max())

s = pd.Series({
    '09-02-2020': 2,
    '09-03-2020': 1,
    '09-06-2020': 5,
    '09-07-2020': 1
})

df.index = pd.DatetimeIndex(df.index)

df = df.reindex(idx, fill_value=np.nan)
df

## Inpute Missing Values

In [ ]:
df

In [ ]:
_ = df['Adjusted_Close'].rolling(200, center=True, min_periods=0).mean().plot()

In [ ]:
df['Adjusted_Close'].interpolate(method='polynomial', order=3, limit_direction='both', inplace=True)
_ = df['Adjusted_Close'].plot()
_ = df['Adjusted_Close'].rolling(200, center=True, min_periods=0).mean().plot()


In [ ]:
df

# Plots

## EDA Plot

In [ ]:
df.info()

In [ ]:
plt.rcParams['font.size'] = 18
fig, ax = plt.subplots(figsize = (20,10))

_ = ax.plot(df.index, df.Adjusted_Close, label = f'{stock_name}')

_ = ax.legend()
_ = ax.grid(True)
ax.set_ylabel('Adjusted Close')
ax.set_title(f'Detecting Pump and Dump Time Series Signature')

_ = plt.show();

# LOWESS Smoothing

In [ ]:
# Compute LOWESS 

# ------- Select variables -------
# y values for both
# y=df['roberta15lg'].values
y=df['Adjusted_Close'].values

# x values for Linear Regression
# X=df['X3 distance to the nearest MRT station'].values.reshape(-1,1) # Note, we need X to be a 2D array, hence reshape
# x values for LOWESS
# x=df['sentence_no'].values 
x=np.arange(df.shape[0])

# ------- Linear Regression -------
# Define and fit the model
# model1 = LinearRegression()
# LR = model1.fit(X, y)

# Predict a few points with Linear Regression model for the grpah
# Create 20 evenly spaced points from smallest X to largest X
# x_range = np.linspace(X.min(), X.max(), 20) 
# Predict y values for our set of X values
# y_range = model1.predict(x_range.reshape(-1, 1))


# ------- LOWESS -------
# Generate y_hat values using lowess, try a couple values for hyperparameters
lowess40 = 1/40
lowess20 = 1/20
lowess10 = 1/10

y_hat40 = lowess(y, x, frac=lowess40)
y_hat20 = lowess(y, x, frac=lowess20)
y_hat10 = lowess(y, x, frac=lowess10)

# Save into 
# df['LOWESS_frac'] = y_hat[:,1].tolist()
df['lowess40'] = y_hat40[:,1].tolist()
df['lowess20'] = y_hat20[:,1].tolist()
df['lowess10'] = y_hat10[:,1].tolist()
df.head()

In [ ]:
# Verify numpy arrays same shape

print(f'x.shape: {x.shape}')
print(f'y.shape: {y.shape}')
print(f'y_hat40.shape: {y_hat40.shape}')
print(f'y_hat20.shape: {y_hat20.shape}')
print(f'y_hat10.shape: {y_hat10.shape}')




In [ ]:
# Interpolate missing data

print(f'x nan count: {pd.Series(x).isna().sum()}')
print(f'y nan count: {pd.Series(y).isna().sum()}')
print(f'y_hat40 nan count: {pd.Series(y_hat40[:,0]).isna().sum()}')
print(f'y_hat20 nan count: {pd.Series(y_hat20[:,0]).isna().sum()}')
print(f'y_hat10 nan count: {pd.Series(y_hat10[:,0]).isna().sum()}')

In [ ]:
# Create a scatter plot

# _ = fig = px.scatter(df, x='sentence_no', y=selected_model.value, custom_data=['text_raw'], opacity=0.3, color_discrete_sequence=['black']) # , size=1)
# _ = fig = px.scatter(df, x='Date', y=['Adjusted_Close'], custom_data=['Open','High','Low','Close'], opacity=0.9) # , color_discrete_sequence=['black']) # , size=1)
_ = fig = px.scatter(df, x=df.index, y=['Adjusted_Close'], custom_data=['Open','High','Low','Close'], opacity=0.9) # , color_discrete_sequence=['black']) # , size=1)
# _ = fig = px.line(df, x=df.index, y=df.Adjusted_Close, custom_data=['Open','High','Low','Close'], opacity=0.9) # , color_discrete_sequence=['black']) # , size=1)

# Add the prediction line
# fig.add_traces(go.Scatter(x=x_range, y=y_range, name='Linear Regression', line=dict(color='limegreen')))
_ = fig.add_traces(go.Scatter(x=df.index, y=df['lowess40'], name=f'LOWESS, frac={lowess40}', line=dict(color='red')))
_ = fig.add_traces(go.Scatter(x=df.index, y=df['lowess20'], name=f'LOWESS, frac={lowess20}', line=dict(color='orange')))
_ = fig.add_traces(go.Scatter(x=df.index, y=df['lowess10'], name=f'LOWESS, frac={lowess10}', line=dict(color='yellow')))
# _ = fig.add_traces(go.Scatter(x=df['Date'], y=y_hat2[:,1], name='LOWESS, frac=1/30', line=dict(color='orange')))

# _ = fig.add_traces(go.Scatter(x=df['sentence_no'], y=df['lowess20'], name='LOWESS, frac=1/20', line=dict(color='red')))
# _ = fig.add_traces(go.Scatter(x=df['sentence_no'], y=df['lowess30'], name='LOWESS, frac=1/30', line=dict(color='orange')))

# _ = fig.add_traces(px.scatter(df, x='sentence_no', y='lowess20', custom_data=['text_raw'])) #, name='LOWESS, frac=1/20', line=dict(color='red')))
# _ = fig.add_traces(px.scatter(df, x='sentence_no', y='lowess30', custom_data=['text_raw'])) #, name='LOWESS, frac=1/30', line=dict(color='orange')))


# _ = fig.add_traces(go.Scatter(x=df['sentence_no'], y=df['sma'], name=f'SMA, {selected_sma_window.value}%', line=dict(color='green'), opacity=0.1)) # , opacity=0.4, color_discrete_sequence=['green']))

# Change chart background color
# _ = fig.update_layout(dict(plot_bgcolor = 'white'))


# Change chart background color
_ = fig.update_layout(dict(plot_bgcolor = 'white'))

# Update axes lines
_ = fig.update_xaxes(showgrid=True, 
                     gridwidth=1, 
                     gridcolor='lightgrey',
                     zeroline=True, 
                     zerolinewidth=1, 
                     zerolinecolor='lightgrey',
                     showline=True,
                     linewidth=1, 
                     linecolor='black',
                     showspikes = True,
                     spikemode  = 'across',
                     spikesnap = 'cursor')

_ = fig.update_yaxes(showgrid=True,
                     gridwidth=1,
                     gridcolor='lightgrey',
                     zeroline=True,
                     zerolinewidth=1,
                     zerolinecolor='lightgrey',
                     showline=True,
                     linewidth=1,
                     linecolor='black')

# Set figure title
# title_str = global_vars.corpus_titles_dt[selected_text.value][0]
title_full_str = f'{stock_name} Pump and Dump Signature (LOWESS frac={LOWESS_FRAC:.2f})'

# Update marker size
_ = fig.update_traces(marker=dict(size=1))

# title_str = f'Sentiment Arc: {title_str}<br>{selected_model.value} with {selected_sma_window.value}% SMA'

_ = fig.update_layout(
    plot_bgcolor='white',
    title=dict(text=title_full_str, y=0.95, x=0.5, xanchor='center', yanchor='bottom'), # font=dict(color='black')),
    xaxis_title="Date",
    yaxis_title='Adjusted Close',
    xaxis=dict(showgrid=True), 
    yaxis=dict(showgrid=True),
    showlegend = True,
    hovermode  = 'x',
    legend=dict(
    title = "Models:",
    orientation="h",
    xanchor="center",
    yanchor="bottom",
    y=-0.3, # 0.99,
    x=0.5 # x=0.01
))

_ = fig.update_traces(
    hovertemplate="<br>".join([
        "Date: %{x}",
        "Closing: %{y}",
        "Open: %{customdata[0]}",
        "High: %{customdata[1]}",
        "Low: %{customdata[2]}",
        "Close: %{customdata[3]}",
    ])
)

fig.show()

# Detect Peaks and Valleys

In [ ]:
# https://github.com/jankoslavic/py-tools/blob/master/findpeaks/findpeaks.py

def findpeaks(data, spacing=1, limit=None):
    """Finds peaks in `data` which are of `spacing` width and >=`limit`.
    :param data: values
    :param spacing: minimum spacing to the next peak (should be 1 or more)
    :param limit: peaks should have value greater or equal
    :return:
    """
    ln = data.size
    x = np.zeros(ln+2*spacing)
    x[:spacing] = data[0]-1.e-6
    x[-spacing:] = data[-1]-1.e-6
    x[spacing:spacing+ln] = data
    peak_candidate = np.zeros(ln)
    peak_candidate[:] = True
    for s in range(spacing):
        start = spacing - s - 1
        h_b = x[start : start + ln]  # before
        start = spacing
        h_c = x[start : start + ln]  # central
        start = spacing + s + 1
        h_a = x[start : start + ln]  # after
        peak_candidate = np.logical_and(peak_candidate, np.logical_and(h_c > h_b, h_c > h_a))

    ind = np.argwhere(peak_candidate)
    ind = ind.reshape(ind.size)
    if limit is not None:
        ind = ind[data[ind] > limit]
    return ind

In [ ]:
df.info()

In [ ]:
# https://github.com/jankoslavic/py-tools/blob/master/findpeaks/Findpeaks%20example.ipynb

_ = plt.figure(figsize=(20,10))

# List of (x,y) coordinates of peaks and valleys
peaks_xy_ls = []
valleys_xy_ls = []

n = 80
m = 20
limit = 0
spacing = 3
# t = np.linspace(0., 1, n)
# x = np.zeros(n)
# np.random.seed(0)
# phase = 2 * np.pi * np.random.random(m)
# for i in range(m):
#     x += np.sin(phase[i] + 2 * np.pi * t * i)

x = df['lowess40'].values
t = df.index.values

# Plot SMA
_ = plt.plot(t, x, alpha=0.3)

# Plot LOWESS
# _ = plt.plot(y_hat2[:,0], y_hat2[:,1])
_ = plt.plot(df.index, df['lowess40'])

# Find Peaks
# peaks = findpeaks(y_hat2[:,1]) # , spacing=spacing, limit=limit)
peaks = findpeaks(df['lowess40'].values) # , spacing=spacing, limit=limit)

# Find Valleys
# valleys = findpeaks(-y_hat2[:,1])
valleys = findpeaks(df['lowess40'].mul(-1.0).values)

# _ = plt.axhline(limit, color='r')
_ = plt.plot(t[peaks], x[peaks], 'g^', markersize=15)
peaks_xy_ls = list(zip(t[peaks],x[peaks]))
_ = plt.plot(t[valleys], x[valleys], 'rv', markersize=20)
valleys_xy_ls = list(zip(t[valleys],x[valleys]))

# Set figure title
# title_str = global_vars.corpus_titles_dt[selected_text.value][0]
title_all_str = f'{stock_name} Pump and Dump (LOWESS {lowess40:.2f})'
peak_str = f'Peaks: min value {limit}, min spacing {spacing} points'

_ = plt.title(f'{title_all_str}\n{peak_str}')
_ = plt.grid(True)
plt.show()


In [ ]:
df.info()

In [ ]:
from scipy.signal import find_peaks

In [ ]:
ts = pd.to_datetime(["2020-01-01", "2020-02-01", "2020-02-02", "2020-02-03", "2020-02-03 12:45:00", "2020-03-01", "2020-04-01"])
ys = [0, 1, 4, 3, 2.7, 2, 1]
# df = pd.DataFrame({'Timestamp':ts, 'Value':ys})
# print(df)

_ = plt.figure(figsize=(20,10))

idx, properties = find_peaks(df['Adjusted_Close'], prominence=0.1, width=10)
# introduce some shorthands:
l = properties["left_ips"]
r = properties["right_ips"]
p = properties["prominences"]
w = properties["widths"]
wh = properties["width_heights"]

peak_x = df.index[idx].values
peak_y = df['Adjusted_Close'][idx].values

print(f"Found a peak at {idx} (index), \ni.e. at {peak_x} (Timestamp) \nwith height {peak_y} (Adjusted_Close)")
print(f"Half of the peak maximum can be found at {wh} (Adjusted_Close) \nand has a width of {w} (Index)!")
print(f"The width starts at {l} (index) \nand goes to {r} (index) == ?? (INTERPOLATION OF INDEX REQUIRED!)")

fig,ax = plt.subplots()  # figsize=(10,3)) 
_ = plt.figure(figsize=(20,10))

_ = ax.plot(df.index,df['Adjusted_Close'], marker="o")

_ = ax.scatter(peak_x, peak_y, marker="x", s=20**2,color="r")

_ = ax.vlines(x=peak_x, ymin=peak_y - p, ymax = peak_y, color = "C1")
# ax.axhline(wh,color="g",linestyle="dashed")

_ = fig.autofmt_xdate()
ax.grid()
plt.tight_layout()
plt.show()

In [ ]:
range_adj_close = df.Adjusted_Close.max() - df.Adjusted_Close.min()
print(f'Range of Adjusted Close: {range_adj_close}')

In [ ]:
df.head()

In [ ]:
df.resample('D')

In [ ]:
df.info()

In [ ]:
ts = pd.to_datetime(["2020-01-01", "2020-02-01", "2020-02-02", "2020-02-03", "2020-02-03 12:45:00", "2020-03-01", "2020-04-01"])
ys = [0, 1, 4, 3, 2.7, 2, 1]
df = pd.DataFrame({'Timestamp':ts, 'Value':ys})
print(df)

idx, properties = find_peaks(df['lowess40'], prominence=0.1, width=0.1)
# introduce some shorthands:
l = properties["left_ips"]
r = properties["right_ips"]
p = properties["prominences"]
w = properties["widths"]
wh = properties["width_heights"]

peak_x = df['Timestamp'][idx].values
peak_y = df['Average_Cl'][idx].values

print(f"Found a peak at {idx} (index), i.e. at {peak_x} (Timestamp) with height {peak_y} (Value)")
print(f"Half of the peak maximum can be found at {wh} (Value) and has a width of {w} (Index)!")
print(f"The width starts at {l} (index) and goes to {r} (index) == ?? (INTERPOLATION OF INDEX REQUIRED!)")

fig,ax = plt.subplots(figsize=(10,3))    
ax.plot(df['Timestamp'],df['Value'], marker="o")

ax.scatter(peak_x, peak_y, marker="x", s=20**2,color="r")

ax.vlines(x=peak_x, ymin=peak_y - p, ymax = peak_y, color = "C1")
ax.axhline(wh,color="g",linestyle="dashed")

fig.autofmt_xdate()
ax.grid()
plt.tight_layout()
plt.show()

## Filter Noisy Peaks/Valleys

In [ ]:
# TODO: Filter out peak/valley and valley/peak pairs 
#         if local(max-min) < (threshold%) * global(max-min)

# OPTIONAL: These noisy peaks are downranked on downstream tasks
#           This functionality only justified for demanding downstream high-speed and/or big data processing

# Extract Hills

In [ ]:
# Merge and Sort

cruxes_xy_ls = peaks_xy_ls + valleys_xy_ls
cruxes_xy_ls.sort(key=lambda tup: tup[0])
cruxes_xy_ls

In [ ]:
len(cruxes_xy_ls)

In [ ]:
# extract tuples (low, high, low) for each hill
# slide 3 wide window over time series
#   if window pattern matches LOW-HIGH-LOW save

hills_ls = []

crux_prev_fl = -99

ptr_end = len(cruxes_xy_ls) - 3  # Window is 3 elements wide

for i, window_ptr in enumerate(range(0, ptr_end)):
  print(f'Processing Window Step #{i}:')
  val0 = cruxes_xy_ls[i][1]
  val1 = cruxes_xy_ls[i+1][1]
  val2 = cruxes_xy_ls[i+2][1]
  print(f'Window: [{val0:.2f}, {val1:.2f}, {val2:.2f}]')
  if (val0 < val1) & (val1 > val2):
    print(f'  Hill at Step = {i}')
    hills_ls.append([val0, val1, val2])


In [ ]:
hills_ls

# Determine Half-Width & Half-Height of each Hill

In [ ]:
from scipy.signal import chirp, find_peaks, peak_widths, peak_prominences

In [ ]:
df.info()

In [ ]:
x

## Find Half and Full Widths

In [ ]:
peaks_lowess, _ = find_peaks(df.LOWESS_frac.values)
peaks_raw, _ = find_peaks(df.Adjusted_Close.values)

# x assigned with LOWESS smoothing above

# Get half/full peaks for LOWESS smoothed
print('HALF WIDTHS ----------')
results_half_tup = peak_widths(x, peaks_lowess, rel_height=0.5)
half_widths_np = results_half_tup[0]  # widths
half_widths_np

print('\n')

print('FULL WIDTHS ----------')
results_full_tup = peak_widths(x, peaks_lowess, rel_height=1)
full_widths_np = results_full_tup[0]
full_widths_np

# Get half/full peaks for Raw Time Series
print('HALF WIDTHS ----------')
results_half_tup = peak_widths(x, peaks_raw, rel_height=0.5)
half_widths_np = results_half_tup[0]  # widths
half_widths_np

print('\n')

print('FULL WIDTHS ----------')
results_full_tup = peak_widths(x, peaks_raw, rel_height=1)
full_widths_np = results_full_tup[0]
full_widths_np

In [ ]:
results_half_tup[2]

In [ ]:
half_widths_np.shape

print('\n')

full_widths_np.shape

In [ ]:
plt.plot(x)

plt.plot(peaks, x[peaks], "x")

plt.hlines(*results_half_tup[1:], color="C2")

plt.hlines(*results_full_tup[1:], color="C3")

plt.show()

## Find Peak Prominences

In [ ]:
x

In [ ]:
# https://stackoverflow.com/questions/53778703/python-scipy-signal-peak-widths-absolute-heigth-fft-3db-damping

# Find peaks
# peaks, _ = find_peaks(x)
# prominences, left_bases, right_bases = peak_prominences(x, peaks)

# Calculate widths at x[peaks] - offset * rel_height
# widths, h_eval, left_ips, right_ips = peak_widths(
#     x, peaks, 
#     rel_height=1,
#     prominence_data=(offset, left_bases, right_bases)
# )

prominences, left_bases, right_bases = peak_prominences(x, peaks)
type(left_bases)
left_bases.shape
left_bases

In [ ]:
peaks, _ = find_peaks(x)

# prominences, left_bases, right_bases = peak_prominences(x, peaks)

prominences_np = peak_prominences(x, peaks)[0]

prominences_np

In [ ]:
peaks, _ = find_peaks(x)

prominences, left_bases, right_bases = peak_prominences(x, peaks)

prominences_np = peak_prominences(x, peaks)[0]

prominences_np

In [ ]:
prominences.shape

In [ ]:
for i in range(left_bases.shape[0]):
  print(f'Prominence #{i}:\n  Left: {left_bases[i]}\n Right: {right_bases[i]}\nHeigth: {right_bases[i]-left_bases[i]}\n') 


In [ ]:
prominences_np.shape

In [ ]:
contour_heights = x[peaks] - prominences_np

_ = plt.plot(x)

_ = plt.plot(peaks, x[peaks], "x")

_ = plt.vlines(x=peaks, ymin=contour_heights, ymax=x[peaks])

plt.show()

In [ ]:
vlines_ls = list(zip(peaks, contour_heights, x[peaks], x[peaks]-contour_heights))

In [ ]:
print(vlines_ls[0])

In [ ]:
# Filter out Prominences without corresponding full/half-width ranges

print(f'Prominences Count: {prominences_np.shape[0]}')
print(f'Half-Width Count: {half_widths_np.shape[0]}')
print(f'Full-Width Count: {full_widths_np.shape[0]}')

In [ ]:
prominences_np[0]

In [ ]:
_ = plt.hlines(*results_half_tup[1:], color="C2")
_ = plt.vlines(x=peaks, ymin=contour_heights, ymax=x[peaks])


In [ ]:
results_half_tup

In [ ]:
results_full_tup

## Select Most Import Prominence/Half-Width Pairs

In [ ]:
ts = pd.to_datetime(["2020-01-01", "2020-02-01", "2020-02-02", "2020-02-03", "2020-02-03 12:45:00", "2020-03-01", "2020-04-01"])
ys = [0, 1, 4, 3, 2.7, 2, 1]
df = pd.DataFrame({'Timestamp':ts, 'Value':ys})
print(df)

idx, properties = find_peaks(df['Value'], prominence=0.1, width=0.1)
# introduce some shorthands:
l = properties["left_ips"]
r = properties["right_ips"]
p = properties["prominences"]
w = properties["widths"]
wh = properties["width_heights"]

peak_x = df['Timestamp'][idx].values
peak_y = df['Value'][idx].values

print(f"Found a peak at {idx} (index), i.e. at {peak_x} (Timestamp) with height {peak_y} (Value)")
print(f"Half of the peak maximum can be found at {wh} (Value) and has a width of {w} (Index)!")
print(f"The width starts at {l} (index) and goes to {r} (index) == ?? (INTERPOLATION OF INDEX REQUIRED!)")

fig,ax = plt.subplots(figsize=(10,3))    
ax.plot(df['Timestamp'],df['Value'], marker="o")

ax.scatter(peak_x, peak_y, marker="x", s=20**2,color="r")

ax.vlines(x=peak_x, ymin=peak_y - p, ymax = peak_y, color = "C1")
ax.axhline(wh,color="g",linestyle="dashed")

fig.autofmt_xdate()
ax.grid()
plt.tight_layout()
plt.show()

In [ ]:
# Create list of vline tuples

vlines_ls = list(zip(peaks, contour_heights, x[peaks], x[peaks]-contour_heights))
print(vlines_ls[0])

In [ ]:
vlines_ls.sort(key=lambda tup: tup[3], reverse=True)
vlines_ls

In [ ]:
# Create list of hline tuples



In [ ]:
# Combine prominence and half/full peak widths

peaks_df = pd.DataFrame({'prominence':prominences_np, 'half_width':half_widths_np, 'full_width':full_widths_np})
peaks_df.head()
peaks_df.info()

In [ ]:
# Compute PnD Radio Statistics

peaks_df['prom2halfwidth'] = peaks_df['prominence']/peaks_df['half_width']
peaks_df['prom2fullwidth'] = peaks_df['prominence']/peaks_df['full_width']
peaks_df.head()
peaks_df.info()

In [ ]:
# Create DataFrame of Peak statistics

peaks_df['prom2halfwidth_rank'] = peaks_df['prom2halfwidth'].rank(ascending=False)
peaks_df['prom2fullwidth_rank'] = peaks_df['prom2fullwidth'].rank(ascending=False)
peaks_df.head()
peaks_df.info()

# Plot

## Static Matplotlib

In [ ]:
df.info()

In [ ]:
plt.rcParams['font.size'] = 18
fig, ax = plt.subplots(figsize = (20,10))

_ = ax.plot(df['Date'], df['Adjusted_Close'], label = 'Adj Close')
_ = ax.plot(df['Date'], df['LOWESS_frac'], label = f'LOWESS, frac={LOWESS_FRAC:.2f}')

_ = ax.legend(loc='best')
_ = ax.grid(True)
ax.set_ylabel('Adjusted Close')
ax.set_title(f'Detecting Pump and Dump Time Series Signature')

_ = plt.show();

## Interactive Plotly

In [ ]:
# Create a scatter plot

# _ = fig = px.scatter(df, x='sentence_no', y=selected_model.value, custom_data=['text_raw'], opacity=0.3, color_discrete_sequence=['black']) # , size=1)
_ = fig = px.scatter(df, x='Date', y=['Adjusted_Close'], custom_data=['Open','High','Low','Close'], opacity=0.9) # , color_discrete_sequence=['black']) # , size=1)

# Add the prediction line
# fig.add_traces(go.Scatter(x=x_range, y=y_range, name='Linear Regression', line=dict(color='limegreen')))
_ = fig.add_traces(go.Scatter(x=df['Date'], y=df['LOWESS_frac'], name=f'LOWESS, frac={LOWESS_FRAC}', line=dict(color='red')))
# _ = fig.add_traces(go.Scatter(x=df['Date'], y=y_hat2[:,1], name='LOWESS, frac=1/30', line=dict(color='orange')))

# _ = fig.add_traces(go.Scatter(x=df['sentence_no'], y=df['lowess20'], name='LOWESS, frac=1/20', line=dict(color='red')))
# _ = fig.add_traces(go.Scatter(x=df['sentence_no'], y=df['lowess30'], name='LOWESS, frac=1/30', line=dict(color='orange')))

# _ = fig.add_traces(px.scatter(df, x='sentence_no', y='lowess20', custom_data=['text_raw'])) #, name='LOWESS, frac=1/20', line=dict(color='red')))
# _ = fig.add_traces(px.scatter(df, x='sentence_no', y='lowess30', custom_data=['text_raw'])) #, name='LOWESS, frac=1/30', line=dict(color='orange')))


# _ = fig.add_traces(go.Scatter(x=df['sentence_no'], y=df['sma'], name=f'SMA, {selected_sma_window.value}%', line=dict(color='green'), opacity=0.1)) # , opacity=0.4, color_discrete_sequence=['green']))

# Change chart background color
# _ = fig.update_layout(dict(plot_bgcolor = 'white'))


# Change chart background color
_ = fig.update_layout(dict(plot_bgcolor = 'white'))

# Update axes lines
_ = fig.update_xaxes(showgrid=True, 
                     gridwidth=1, 
                     gridcolor='lightgrey',
                     zeroline=True, 
                     zerolinewidth=1, 
                     zerolinecolor='lightgrey',
                     showline=True,
                     linewidth=1, 
                     linecolor='black',
                     showspikes = True,
                     spikemode  = 'across',
                     spikesnap = 'cursor')

_ = fig.update_yaxes(showgrid=True,
                     gridwidth=1,
                     gridcolor='lightgrey',
                     zeroline=True,
                     zerolinewidth=1,
                     zerolinecolor='lightgrey',
                     showline=True,
                     linewidth=1,
                     linecolor='black')

# Set figure title
# title_str = global_vars.corpus_titles_dt[selected_text.value][0]
title_full_str = f'{stock_name} Pump and Dump Signature (LOWESS frac={LOWESS_FRAC:.2f})'

# Update marker size
_ = fig.update_traces(marker=dict(size=1))

# title_str = f'Sentiment Arc: {title_str}<br>{selected_model.value} with {selected_sma_window.value}% SMA'

_ = fig.update_layout(
    plot_bgcolor='white',
    title=dict(text=title_full_str, y=0.95, x=0.5, xanchor='center', yanchor='bottom'), # font=dict(color='black')),
    xaxis_title="Date",
    yaxis_title='Adjusted Close',
    xaxis=dict(showgrid=True), 
    yaxis=dict(showgrid=True),
    showlegend = True,
    hovermode  = 'x',
    legend=dict(
    title = "Models:",
    orientation="h",
    xanchor="center",
    yanchor="bottom",
    y=-0.3, # 0.99,
    x=0.5 # x=0.01
))

_ = fig.update_traces(
    hovertemplate="<br>".join([
        "Date: %{x}",
        "Closing: %{y}",
        "Open: %{customdata[0]}",
        "High: %{customdata[1]}",
        "Low: %{customdata[2]}",
        "Close: %{customdata[3]}",
    ])
)

fig.show()

# **[END OF NOTEBOOK]**

In [ ]:
from pandas.io.formats import style
# Review first 5 pump_indexes

max_pumpindx_dates_dt = {}
pump_indx_max = 5

styles = ['-','--','-.',':','-']
linewidths = [2, 1, 4]
for apump_indx in range(pump_indx_max):
  _ = df[df['pump_index'] == apump_indx]['avg_price'].plot() # alpha=0.5)

_ = plt.legend(loc='best')
_ = plt.grid(True)
_ = plt.title('Raw Avg Prices')
plt.show();

for apump_indx in range(pump_indx_max):
  amax_date = df[df['pump_index'] == apump_indx]['date'].max()
  print(f'pump_indx #{apump_indx} amax_date: {amax_date}')
  max_pumpindx_dates_dt[apump_indx] = amax_date
  _ = df[df['pump_index'] == apump_indx]['avg_price'].rolling(100, center=True, min_periods=0).mean().plot(label=f'win=100, pindx={apump_indx}', alpha=0.5)
  _ = plt.legend(loc='best')
  _ = df[df['pump_index'] == apump_indx]['avg_price'].rolling(200, center=True, min_periods=0).mean().plot(label=f'win=200', alpha=0.5)
  _ = df[df['pump_index'] == apump_indx]['avg_price'].rolling(200, center=True, min_periods=0).mean().plot(label=f'win=300', alpha=0.5)

_ = plt.legend(loc='best')
_ = plt.grid(True)
_ = plt.title('SMA 10 Smoothed Avg Prices')
plt.show();

## Plotly

In [ ]:
# settings
pd.options.plotting.backend = "plotly"

# sample dataframe of a wide format
np.random.seed(4); cols = list('abc')
X = np.random.randn(50,len(cols))  
adf=pd.DataFrame(X, columns=cols)
adf.iloc[0]=0; adf=adf.cumsum()

# plotly figure
adf.plot(template = 'plotly_white')

# setback
pd.options.plotting.backend = 'matplotlib'

In [ ]:
df.info()

In [ ]:
"""

apump_index = 3

# settings
pd.options.plotting.backend = "plotly"

# sample dataframe of a wide format
np.random.seed(4); cols = list('abc')
X = np.random.randn(50,len(cols))  
# df=pd.DataFrame(X, columns=cols)
# df.iloc[0]=0; df=df.cumsum()

# plotly figure
df[df['pump_index'] == apump_index]['avg_price'].plot(template = 'plotly_white')


# Zoom around the Max Peak
ahalf_win = 20

aindx_max = df[df['pump_index'] == apump_index]['avg_price'].idxmax()
aval_max = df[df['pump_index'] == apump_index]['avg_price'].max()

print(f'aval_max: {aval_max} at idxmax: {aindx_max}')

# plotly figure
df.iloc[aindx_max - ahalf_win:aindx_max + ahalf_win]['avg_price'].plot(template = 'plotly_white')

# set back
pd.options.plotting.backend = "matplotlib"

""";

In [ ]:
df['pump_index'].max()

In [ ]:
apump_indx_ls = [1,2,3,4,5]
apump_indx_ls = list(range(100,105))

# Get some data
# df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/finance-charts-apple.csv')

plt.plot()
for i, apump_indx in enumerate(apump_indx_ls):
  max_val = df[df['pump_index']==apump_indx]['avg_price'].max()
  max_indx = df[df['pump_index']==apump_indx]['avg_price'].idxmax() 
  max_date = df.iloc[max_indx]['date']

  atitle_str = f'Pump_Index={apump_indx}: Max={max_val} on date={max_date} (index={max_indx})'

  fig = px.line(df[df['pump_index']==apump_indx], x='date', y=['avg_price'])
  fig.add_vline(x=max_date, line_width=3, line_dash="dot", line_color="red")
  # fig.update_layout(title_font=dict(size=30))
  fig.update_layout(title_text=atitle_str, title_x=0.5)

  # fig.update_layout(
  #   showlegend=False,
  #   font={'size': 11},
  #   title={'text': atitle_str, 'font': {'size': 14}, title_x=0.5}
  # )


 

# Test Sandbox

In [ ]:
"""

pump_indx_max = 5

# data set 1
# x = np.linspace(0, 9, 10)
# y = x

# data set 2
# df = pd.DataFrame(np.column_stack([x*0.5, y]), columns=["x", "y"])

for apump_indx in range(pump_indx_max):
  amax_date = df[df['pump_index'] == apump_indx]['date'].max()
  print(f'pump_indx #{apump_indx} amax_date: {amax_date}')
  max_pumpindx_dates_dt[apump_indx] = amax_date
                       
  fig = go.Figure(px.scatter(df[df['pump_index'==apump_indx]], x="date", y="avg_price"))
  fig.add_trace(go.Scatter(x=x, y=y))

fig.show()

""";

In [ ]:
def method1(df):
  D = {}
  [D.setdefault(k, []).append(v) for k, v in zip(df['name'].to_numpy(), df['val'].to_numpy())]
  return D

In [ ]:
from collections import defaultdict

pump_dates_ls = []

d = defaultdict(list)
first_fl = True
pump_cur_int = 0
date_end_dt = 

for _, row in df.iterrows():
  pump_last_int = pump_cur_int
  pump_cur_int = row['pump_index']
  if (pump_last_int != pump_cur_int) & (pump_cur_int == 1):
    date_start_dt = row['date']
  elif (pump_last_int != pump_cur_int) & (pump_cur_int == 0):
    date_end_dt = row['date']
  else:
    continue


  else:
    d[row['name']].append(row['val'])
    
dict(d)

In [ ]:
transitions = df.groupby('userid').category.diff().fillna(0).ne(0)
>>> x[transitions].groupby('userid').agg(list)